Pg.No: 95 - 
# Left Outer Join

In [1]:
def parseUser(data):
    return tuple(data.split('|'))
user_data = sc.textFile('./DataSet/users.txt').map(parseUser)
user_data.take(3)

[('u1', 'UT'), ('u2', 'GA'), ('u3', 'CA')]

In [2]:
def parseTransactions(data):
    dataList = data.split('|')
    return (dataList[0], dataList[1], dataList[2], int(dataList[3]), int(dataList[4]))
transactions_data = sc.textFile('./DataSet/transactions.txt').map(parseTransactions)
transactions_data.take(3)

[('t1', 'p3', 'u1', 1, 300),
 ('t2', 'p1', 'u2', 1, 100),
 ('t3', 'p1', 'u1', 1, 100)]

In [3]:
userPairRDD = user_data.map(lambda iter1: (iter1[0],('L', iter1[1])))

In [4]:
userPairRDD.take(4)

[('u1', ('L', 'UT')),
 ('u2', ('L', 'GA')),
 ('u3', ('L', 'CA')),
 ('u4', ('L', 'CA'))]

In [5]:
transactionPairRDD = transactions_data.map(lambda iter1: (iter1[2], ('P', iter1[1])))

In [6]:
transactionPairRDD.take(4)

[('u1', ('P', 'p3')),
 ('u2', ('P', 'p1')),
 ('u1', ('P', 'p1')),
 ('u2', ('P', 'p2'))]

In [7]:
utUnionRDD = transactionPairRDD.union(userPairRDD)

In [8]:
utUnionRDD.collect()

[('u1', ('P', 'p3')),
 ('u2', ('P', 'p1')),
 ('u1', ('P', 'p1')),
 ('u2', ('P', 'p2')),
 ('u4', ('P', 'p4')),
 ('u1', ('P', 'p1')),
 ('u1', ('P', 'p4')),
 ('u5', ('P', 'p4')),
 ('u1', ('L', 'UT')),
 ('u2', ('L', 'GA')),
 ('u3', ('L', 'CA')),
 ('u4', ('L', 'CA')),
 ('u5', ('L', 'GA'))]

In [9]:
utGrpbyKey = utUnionRDD.groupByKey()
# groupbykey makes ResultIterable we need to change it datatype to list
# see next statement

In [14]:
utGrpByKeyVal = utGrpbyKey.mapValues(lambda x: list(x))
utGrpByKeyVal.collect()

[('u5', [('P', 'p4'), ('L', 'GA')]),
 ('u2', [('P', 'p1'), ('P', 'p2'), ('L', 'GA')]),
 ('u4', [('P', 'p4'), ('L', 'CA')]),
 ('u1', [('P', 'p3'), ('P', 'p1'), ('P', 'p1'), ('P', 'p4'), ('L', 'UT')]),
 ('u3', [('L', 'CA')])]

In [28]:
def flatMapValue(data):
    products = []
    productList = data[1][: -1]
    location = data[1][-1]
    for product in productList:
        products.append((product[1], location[1]))
    return products
utGrpByKeyVal.map(flatMapValue).flatMap(lambda iter1: iter1).collect()

[('p4', 'GA'),
 ('p1', 'GA'),
 ('p2', 'GA'),
 ('p4', 'CA'),
 ('p3', 'UT'),
 ('p1', 'UT'),
 ('p1', 'UT'),
 ('p4', 'UT')]